In [1]:
import pandas as pd
import pyodbc
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def criar_conexao():
    dados_conexao = (
        'Driver={SQL Server};'
        f'Server=SQLPW90DB03\DBINST3, 1443'
        f'Database=BDintelicanais;'
        'Trusted_Connection=yes;'
    )
    return pyodbc.connect(dados_conexao)


#comando_sql = """select * from TBL_CDO_FISICOS_TABLEAU where DT_ANOMES >= 202311"""
comando_sql = """
select 
	R.COD_REGIONAL, 
	C.DS_CANAL_FINAL, 
    C.DS_GESTAO,
	CASE WHEN C.DS_GESTAO = 'GESTAO REGIONAL' THEN R.COD_REGIONAL 
         WHEN C.DS_GESTAO = 'GESTAO NACIONAL' AND C.DS_CANAL_FINAL like ('%TLV%') THEN 'TLV'
         WHEN C.DS_GESTAO = 'GESTAO NACIONAL' AND C.DS_CANAL_FINAL like ('%WEB%') THEN 'WEB'
         ELSE C.DS_GESTAO END  AS DS_GESTAO_REGIONAL_CANAL,
	case when ds_det_indicador in ('NOVOS CLIENTES','MIG AQUISICAO') then 'AQUISIÇÃO' else 'MIG BASE' end as tipo_det_indicador,
	a.*
--select distinct ds_det_indicador
FROM dbo.TBL_CDO_FISICOS_REAL as A
LEFT OUTER JOIN dbo.TBL_CDO_DE_PARA_REGIONAL AS R ON A.NO_CURTO_TERRITORIO = R.NO_CURTO_TERRITORIO
LEFT OUTER JOIN dbo.TBL_CDO_DE_PARA_CANAL AS C ON A.DS_CANAL_BOV = C.DS_DESCRICAO_CANAL_BOV
where DT_ANOMES >='202401'
	and DS_INDICADOR = 'VL'

"""

conexao = criar_conexao()
df = pd.read_sql(comando_sql, conexao)

df['DT_REFERENCIA'] = pd.to_datetime(df['DT_REFERENCIA'], format='%d/%m/%Y')

df.head()

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\oi066724\AppData\Local\Temp\ipykernel_13316\1540833450.py:4: SyntaxWarning: invalid escape sequence '\D'
  f'Server=SQLPW90DB03\DBINST3, 1443'
C:\Users\oi066724\AppData\Local\Temp\ipykernel_13316\1540833450.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(comando_sql, conexao)


,COD_REGIONAL,DS_CANAL_FINAL,DS_GESTAO,DS_GESTAO_REGIONAL_CANAL,tipo_det_indicador,DS_TIPO,DS_PRODUTO,DS_UGR,DS_GRUPO_INDICADOR,DS_INDICADOR,...,CD_SURVEY,CD_CELULA,DS_VELOCIDADE,DS_NUMERO_PEDIDO,DS_NUMERO_PEDIDO_ORIG,VL_OFERTA,DT_ANOMES,QTD,DS_SEGMENTACAO,DS_CLASS_CELULA
0,RCS,WEB - BOT,GESTAO NACIONAL,WEB,AQUISIÇÃO,REAL,NOVA FIBRA,HC,FISICOS,VL,...,O1015753,54 (PNCE) SECUNDÁRIA,600 MBPS,01850833,None,129.9,202401,1.0,MASSIVO,BRONZE
1,RCS,WEB - BOT,GESTAO NACIONAL,WEB,AQUISIÇÃO,REAL,NOVA FIBRA,HC,FISICOS,VL,...,T1726922,200 (SCR2) SECUNDÁRIA,400 MBPS,01850821,None,109.9,202401,1.0,MASSIVO,BRONZE
2,RCS,WEB - AFILIADOS,GESTAO NACIONAL,WEB,AQUISIÇÃO,REAL,NOVA FIBRA,HC,FISICOS,VL,...,W300387,83 (FNS) SECUNDÁRIA,400 MBPS,01850838,None,89.9,202401,1.0,ALTO VALOR,OURO
3,RCS,WEB - AFILIADOS,GESTAO NACIONAL,WEB,AQUISIÇÃO,REAL,NOVA FIBRA,HC,FISICOS,VL,...,J548889,81 (CBCH) SECUNDÁRIA,600 MBPS,01850850,None,129.9,202401,1.0,ALTO VALOR,OURO
4,RSE,WEB - BOT,GESTAO NACIONAL,WEB,AQUISIÇÃO,REAL,NOVA FIBRA,HC,FISICOS,VL,...,L1559995,34 (RBR) SECUNDÁRIA,600 MBPS,01850809,None,109.9,202401,1.0,MASSIVO,BRONZE


In [3]:
df_vl_real = df.query('DS_INDICADOR == "VL" and DS_TIPO == "REAL" and tipo_det_indicador == "AQUISIÇÃO"')
df_vl_real2 = df_vl_real[['DS_TIPO','DS_PRODUTO','DS_VELOCIDADE','DS_GESTAO_REGIONAL_CANAL','DT_REFERENCIA','QTD']]


df_vl_real2

,DS_TIPO,DS_PRODUTO,DS_VELOCIDADE,DS_GESTAO_REGIONAL_CANAL,DT_REFERENCIA,QTD
0,REAL,NOVA FIBRA,600 MBPS,WEB,2024-01-24,1.0
1,REAL,NOVA FIBRA,400 MBPS,WEB,2024-01-24,1.0
2,REAL,NOVA FIBRA,400 MBPS,WEB,2024-01-24,1.0
3,REAL,NOVA FIBRA,600 MBPS,WEB,2024-01-24,1.0
4,REAL,NOVA FIBRA,600 MBPS,WEB,2024-01-24,1.0
...,...,...,...,...,...,...
190258,REAL,NOVA FIBRA,600 MBPS,RCS,2024-01-02,1.0
190259,REAL,NOVA FIBRA,500 MBPS,RCS,2024-01-02,1.0
190260,REAL,NOVA FIBRA,600 MBPS,TLV,2024-01-02,1.0
190261,REAL,NOVA FIBRA,500 MBPS,RCS,2024-01-02,1.0


# VL Fibra Legado e Nova Fibra

In [4]:
#df_vl_real3 = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL in ("RNN","RSE", "RCS")')
df_vl2 = df_vl_real2[['DS_TIPO','DS_PRODUTO', 'DS_VELOCIDADE','DT_REFERENCIA','QTD']]
df_vl3 = df_vl2.groupby(['DS_TIPO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

df_vl_NF_real_oi= df_vl2.query('DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_oi2 = df_vl_NF_real_oi.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

df_vl_LEG_real_oi = df_vl2.query('DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_oi2 = df_vl_LEG_real_oi.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl3, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='QTD',
             text_auto=True)
fig3.update_layout(title='VL Fibra + Nova Fibra')
fig3.update_xaxes(tickangle=-90)
fig3.show()

fig1 = px.bar(df_vl_NF_real_oi2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='Oi VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()



fig2 = px.bar(df_vl_LEG_real_oi2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig2.update_layout(title='Oi VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()




# RNN

In [5]:
df_vl_real_RNN = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RNN"')
df_vl_real_RNN2 = df_vl_real_RNN.groupby(['DS_TIPO', 'DS_PRODUTO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl_real_RNN2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_PRODUTO',
             text_auto=True)
fig3.update_layout(title='RNN VL')
fig3.update_xaxes(tickangle=-90)
fig3.show()

df_vl_NF_real_RNN = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RNN" and DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_RNN2 = df_vl_NF_real_RNN.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

df_vl_LEG_real_RNN = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RNN" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_RNN2 = df_vl_LEG_real_RNN.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()



fig1 = px.bar(df_vl_NF_real_RNN2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='RNN VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()

df_vl_LEG_real_RNN = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RNN" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_RNN2 = df_vl_LEG_real_RNN.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig2 = px.bar(df_vl_LEG_real_RNN2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig2.update_layout(title='RNN VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()

# RSE

In [6]:
df_vl_real_RSE = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RSE"')
df_vl_real_RSE2 = df_vl_real_RSE.groupby(['DS_TIPO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl_real_RSE2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='QTD',
             text_auto=True)
fig3.update_layout(title='RSE VL')
fig3.update_xaxes(tickangle=-90)
fig3.show()


df_vl_NF_real_RSE = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RSE" and DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_RSE2 = df_vl_NF_real_RSE.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig1 = px.bar(df_vl_NF_real_RSE2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='RSE VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()

df_vl_LEG_real_RSE = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RSE" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_RSE2 = df_vl_LEG_real_RSE.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig2 = px.bar(df_vl_LEG_real_RSE2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig2.update_layout(title='RSE VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()

# RCS

In [7]:
df_vl_real_RCS = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RCS"')
df_vl_real_RCS2 = df_vl_real_RCS.groupby(['DS_TIPO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl_real_RCS2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='QTD',
             text_auto=True)
fig3.update_layout(title='RCS VL')
fig3.update_xaxes(tickangle=-90)
fig3.show()



df_vl_NF_real_RCS = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RCS" and DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_RCS2 = df_vl_NF_real_RCS.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig1 = px.bar(df_vl_NF_real_RCS2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='RCS VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()

df_vl_LEG_real_RCS = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "RCS" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_RCS2 = df_vl_LEG_real_RCS.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig2 = px.bar(df_vl_LEG_real_RCS2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig2.update_layout(title='RCS VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()

# TLV

In [8]:
df_vl_real_TLV = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "TLV"')
df_vl_real_TLV2 = df_vl_real_TLV.groupby(['DS_TIPO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl_real_TLV2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='QTD',
             text_auto=True)
fig3.update_layout(title='TLV VL')
fig3.update_xaxes(tickangle=-90)
fig3.show()


df_vl_NF_real_TLV = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "TLV" and DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_TLV2 = df_vl_NF_real_TLV.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig1 = px.bar(df_vl_NF_real_TLV2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='TLV VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()

df_vl_LEG_real_TLV = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "TLV" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_TLV2 = df_vl_LEG_real_TLV.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig2 = px.bar(df_vl_LEG_real_TLV2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig2.update_layout(title='TLV VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()

# WEB

In [9]:
df_vl_real_WEB = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "WEB"')
df_vl_real_WEB2 = df_vl_real_WEB.groupby(['DS_TIPO', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig3 = px.bar(df_vl_real_WEB2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='QTD',
             text_auto=True)
fig3.update_layout(title='WEB VL')
fig3.update_xaxes(tickangle=-90)
fig3.show()



df_vl_NF_real_WEB = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "WEB" and DS_PRODUTO == "NOVA FIBRA" ')
df_vl_NF_real_WEB2 = df_vl_NF_real_WEB.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig1 = px.bar(df_vl_NF_real_WEB2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True)
fig1.update_layout(title='WEB VL NF')
fig1.update_xaxes(tickangle=-90)
fig1.show()

df_vl_LEG_real_WEB = df_vl_real2.query('DS_GESTAO_REGIONAL_CANAL == "WEB" and DS_PRODUTO == "FIBRA" ')
df_vl_LEG_real_WEB2 = df_vl_LEG_real_WEB.groupby(['DS_TIPO', 'DS_PRODUTO', 'DS_VELOCIDADE', 'DT_REFERENCIA']).agg({'QTD': 'sum'}).reset_index()

fig2 = px.bar(df_vl_LEG_real_WEB2, 
             x='DT_REFERENCIA', 
             y='QTD',
             color='DS_VELOCIDADE',
             text_auto=True,
             )
fig2.update_layout(title='WEB VL LEGADO')
fig2.update_xaxes(tickangle=-90)
fig2.show()